# **Desafio 4 - Trilhas**

## **Introdução**
O seu gerente lhe apresentou uma planilha contendo dados sobre a situação atual dos clientes do banco em alguns estados do Nordeste, a fim de analisar e ajudar na tomada de decisão. No entanto, como é de se esperar, os dados brutos apresentam diversas inconsistências e precisam passar por um processo de limpeza. Então, você, como um bom cientista de dados, sabe que nem tudo são flores: os dados são como petróleo bruto, mas inúteis até serem devidamente refinados para análises e tomadas de decisão.

### **PARTE 1 - LIMPEZA E TRATAMENTO DE DADOS:**

#### Importação de bibliotecas e leitura do dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Leitura do arquivo CSV
url = "https://raw.githubusercontent.com/jefferssonc/Desafio4_Trilhas/refs/heads/main/Trilhas2B-Desafio%204.csv"
df = pd.read_csv(url)

# Visualização inicial
df.head()


 ####  Altere os nomes das colunas para letras minúsculas, utilizando a biblioteca pandas

In [ ]:
df.columns = df.columns.str.lower()
df.columns.tolist()


In [ ]:
print(f"Valores únicos de idade {df['idade'].unique()}\nValores únicos de genero {df['genero'].unique()}\nValores únicos de estado {df['estado'].unique()}")

In [ ]:
# Calcular a porcentagem de NaNs na coluna 'genero'
percentual_nan_genero = df['genero'].isna().mean() * 100

# Exibir o resultado com duas casas decimais
print(f"Porcentagem de valores ausentes (NaN) em 'genero': {percentual_nan_genero:.2f}%")


In [ ]:
#Taxa pequena então ao invés de dropar irei apenas mudar o status das mesmas
df['genero'] = df['genero'].fillna('Não Informado')
print(df['genero'].unique())

In [ ]:
# Definir siglas válidas
siglas_validas = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']

# Calcular o número de registros com siglas inválidas
qtd_invalidas = (df['estado'].apply(lambda x: x not in siglas_validas)).sum()

# Calcular a porcentagem de siglas inválidas
qtd_total = len(df)
porcentagem_invalidas = (qtd_invalidas / qtd_total) * 100

# Exibir o resultado
print(f"Quantidade de siglas inválidas: {qtd_invalidas}")
print(f"Porcentagem de siglas inválidas: {porcentagem_invalidas:.2f}%")


In [ ]:
#Aqui já irei optar por dropar, mas nada impediria de mudar o status da sigla errada, no fim só quero usar outra abordagem além de dropar :)
# Remover registros com siglas inválidas
df = df[df['estado'].isin(siglas_validas)].reset_index(drop=True)
print(df['estado'].unique())


 #### Explorar colunas categóricas com `groupby()` e visualização

In [ ]:
# Análise de colunas categóricas com groupby
print("\nClientes por estado:")
print(df.groupby('estado')['id'].count().sort_values(ascending=False))

print("\nClientes por gênero:")
print(df.groupby('genero')['id'].count())

print("\nPossui cartão por estado:")
print(df.groupby(['estado', 'possui cartao'])['id'].count().unstack())

print("\nRelação entre cliente ativo e saiu:")
print(df.groupby(['ativo', 'saiu'])['id'].count().unstack())




In [ ]:
# Gráficos para análise de colunas categóricas
plt.figure(figsize=(10,6))
sns.countplot(data=df, x='estado', order=df['estado'].value_counts().index)
plt.title('Distribuição de Clientes por Estado')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(8,5))
sns.countplot(data=df, x='genero')
plt.title('Distribuição de Clientes por Gênero')
plt.show()

plt.figure(figsize=(10,6))
sns.countplot(data=df, x='estado', hue='possui cartao', order=df['estado'].value_counts().index)
plt.title('Possui Cartão por Estado')
plt.xticks(rotation=45)
plt.legend(title='Possui Cartão')
plt.show()

plt.figure(figsize=(8,5))
sns.countplot(data=df, x='ativo', hue='saiu')
plt.title('Cliente Ativo vs Churn')
plt.legend(title='SAIU')
plt.show()

 #### Explorar colunas numéricas com `describe()` e boxplots

In [ ]:
print(df.describe())

# Boxplot para detectar outliers
num_cols = df.select_dtypes(include=np.number).columns
plt.figure(figsize=(15,8))
df[num_cols].boxplot(rot=90)
plt.title("Boxplot das Categorias Numéricas")
plt.show()


In [ ]:
# Boxplot para cada categoria(separado) para detectar outliers
colunas_numericas = df.select_dtypes(exclude='object').columns
for col in colunas_numericas:
    plt.figure(figsize=(8,4))
    sns.boxplot(data=df, y=col)
    plt.title(f'Boxplot de {col}')
    plt.show()

 #### Corrigir valores faltantes




In [ ]:
# Verificar valores faltantes
print(f"{'='*15}Antes do tratamento{'='*15}")
print(df.isnull().sum())

# Preencher valores numéricos com a mediana
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Preencher valores categóricos com a moda
colunas_categoricas = df.select_dtypes(include='object').columns
for col in colunas_categoricas:
    df[col] = df[col].fillna(df[col].mode()[0])

# Verificar novamente
print(f"{'='*15}Depois do tratamento{'='*15}")
print(df.isnull().sum())


 #### Tratar outliers (exemplo com IQR)

In [ ]:
# Lista de colunas numéricas que vamos tratar
colunas_numericas = ['pontos', 'idade', 'bens', 'saldo na conta', 'salario anual']

for coluna in colunas_numericas:
    # Calculando Q1, Q3 e IQR
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1

    # Definindo limites superior e inferior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Mediana da coluna para substituição
    mediana = df[coluna].median()

    # Substituir outliers por mediana usando np.where (mais eficiente)
    df[coluna] = np.where(df[coluna] < limite_inferior, mediana, df[coluna])
    df[coluna] = np.where(df[coluna] > limite_superior, mediana, df[coluna])

print("Outliers tratados com sucesso.")

 #### Eliminar duplicados

In [ ]:
df = df.drop_duplicates()
print("Dimensão após remover duplicatas:", df.shape)


 #### Corrigir dados categóricos inconsistentes

In [ ]:
# Corrigir coluna "genero"
df['genero'] = df['genero'].replace({
    'Mas': 'Masculino',
    'M': 'Masculino',
    'Fem': 'Feminino',
    'F': 'Feminino'
})

df['genero'].value_counts()


In [ ]:
print(f"Valores únicos de idade {df['idade'].unique()}\nValores únicos de genero {df['genero'].unique()}\nValores únicos de estado {df['estado'].unique()}")

 #### Mostrar o tamanho do dataset

In [ ]:
print("Shape do dataset:", df.shape)


### **PARTE 2 - ESTATÍSTICA**

#### Média e mediana do saldo (< 40 anos)

In [ ]:
df[df['idade'] < 40]['saldo na conta'].agg(['mean', 'median'])


#### Média e mediana do saldo (> 40 anos)

In [ ]:
df[df['idade'] > 40]['saldo na conta'].agg(['mean', 'median'])


 #### Média e mediana do saldo por status de saída

In [ ]:
df.groupby('saiu')['saldo na conta'].agg(['mean', 'median'])


#### Análise do público que saiu

In [ ]:
# Filtrar os dados para clientes que saíram
clientes_sairam = df[df['saiu'] == 1]

# 1. Exibir as informações dos clientes que saíram (idade, saldo na conta, bens e estado)
informacoes_sairam = clientes_sairam[['genero', 'idade', 'saldo na conta', 'bens', 'estado']]

# 2. Verificar o público predominante (Masculino ou Feminino)
genero_predominante = clientes_sairam['genero'].value_counts().idxmax()

# 3. Calcular a quantidade por sexo (Masculino e Feminino) e seus respectivos percentuais
sexo_count = clientes_sairam['genero'].value_counts()
sexo_percentual = clientes_sairam['genero'].value_counts(normalize=True) * 100

# 4. Calcular a distribuição percentual por estado
estado_percentual = clientes_sairam['estado'].value_counts(normalize=True) * 100

# 5. Calcular a menor e maior idade dos que saíram, tanto Masculino quanto Feminino
menor_idade_masculino = clientes_sairam[clientes_sairam['genero'] == 'Masculino']['idade'].min()
maior_idade_masculino = clientes_sairam[clientes_sairam['genero'] == 'Masculino']['idade'].max()

menor_idade_feminino = clientes_sairam[clientes_sairam['genero'] == 'Feminino']['idade'].min()
maior_idade_feminino = clientes_sairam[clientes_sairam['genero'] == 'Feminino']['idade'].max()

# 6. Calcular a média da idade, saldo na conta e bens, tanto para Masculino quanto para Feminino
media_idade_masculino = clientes_sairam[clientes_sairam['genero'] == 'Masculino']['idade'].mean()
media_idade_feminino = clientes_sairam[clientes_sairam['genero'] == 'Feminino']['idade'].mean()

media_saldo_masculino = clientes_sairam[clientes_sairam['genero'] == 'Masculino']['saldo na conta'].mean()
media_saldo_feminino = clientes_sairam[clientes_sairam['genero'] == 'Feminino']['saldo na conta'].mean()

media_bens_masculino = clientes_sairam[clientes_sairam['genero'] == 'Masculino']['bens'].mean()
media_bens_feminino = clientes_sairam[clientes_sairam['genero'] == 'Feminino']['bens'].mean()

# Exibir os resultados
print(f"Público predominante entre os que saíram: {genero_predominante}")
print("\nInformações dos clientes que saíram (idade, saldo na conta, bens e estado):")
print(informacoes_sairam)

print("\nQuantidade de clientes por sexo e seus percentuais:")
print(sexo_count)
print(sexo_percentual)

print("\nDistribuição percentual por estado:")
print(estado_percentual)

print("\nMenor e maior idade dos que saíram:")
print(f"Masculino: Menor idade = {menor_idade_masculino:.0f}, Maior idade = {maior_idade_masculino:.0f}")
print(f"Feminino: Menor idade = {menor_idade_feminino:.0f}, Maior idade = {maior_idade_feminino:.0f}")

print("\nMédia dos clientes que saíram:")
print(f"Masculino - Idade: {media_idade_masculino:.0f}, Saldo na conta: {media_saldo_masculino:.2f}, Bens: {media_bens_masculino:.2f}")
print(f"Feminino - Idade: {media_idade_feminino:.0f}, Saldo na conta: {media_saldo_feminino:.2f}, Bens: {media_bens_feminino:.2f}")


In [ ]:

# Configurações dos gráficos
sns.set(style="whitegrid")

# 1. Gráfico de distribuição por sexo (Masculino e Feminino)
plt.figure(figsize=(8, 6))
sexo_count.plot(kind='bar', color=['blue', 'orange'])
plt.title('Distribuição por Sexo (Clientes que Saíram)')
plt.xlabel('Sexo')
plt.ylabel('Quantidade')
plt.xticks(rotation=0)
plt.show()

# 2. Gráfico de distribuição percentual por sexo
plt.figure(figsize=(8, 6))
sexo_percentual.plot(kind='bar', color=['blue', 'orange'])
plt.title('Percentual por Sexo (Clientes que Saíram)')
plt.xlabel('Sexo')
plt.ylabel('Percentual (%)')
plt.xticks(rotation=0)
plt.show()

# 3. Gráfico de distribuição percentual por estado
plt.figure(figsize=(10, 6))
estado_percentual.plot(kind='bar', color='lightgreen')
plt.title('Distribuição Percentual por Estado (Clientes que Saíram)')
plt.xlabel('Estado')
plt.ylabel('Percentual (%)')
plt.xticks(rotation=45)
plt.show()

# 4. Gráfico de boxplot para distribuição de idades por gênero
plt.figure(figsize=(8, 6))
sns.boxplot(x='genero', y='idade', data=clientes_sairam)
plt.title('Distribuição de Idades por Gênero (Clientes que Saíram)')
plt.xlabel('Gênero')
plt.ylabel('Idade')
plt.show()

# 5. Gráfico de barras para a média do saldo na conta por gênero
media_saldo = [media_saldo_masculino, media_saldo_feminino]
generos = ['Masculino', 'Feminino']

plt.figure(figsize=(8, 6))
plt.bar(generos, media_saldo, color=['blue', 'orange'])
plt.title('Média do Saldo na Conta por Gênero (Clientes que Saíram)')
plt.xlabel('Gênero')
plt.ylabel('Média do Saldo na Conta')
plt.show()

# 6. Gráfico de barras para a média dos bens por gênero
media_bens = [media_bens_masculino, media_bens_feminino]

plt.figure(figsize=(8, 6))
plt.bar(generos, media_bens, color=['blue', 'orange'])
plt.title('Média dos Bens por Gênero (Clientes que Saíram)')
plt.xlabel('Gênero')
plt.ylabel('Média dos Bens')
plt.show()


In [ ]:
# Comparação entre saíram e não saíram por gênero
plt.figure(figsize=(8,6))
sns.countplot(data=df, x='genero', hue='saiu', palette='Set2')
plt.title('Comparação de Gênero entre Clientes que Saíram e Permaneceram')
plt.xlabel('Gênero')
plt.ylabel('Quantidade')
plt.legend(title='Saiu', labels=['Não', 'Sim'])
plt.show()


In [ ]:
#Gráfico de pizza para mostrar distribuição por estado dos que saíram
plt.figure(figsize=(7,7))
estado_percentual.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=sns.color_palette("pastel"))
plt.title('Distribuição por Estado dos Clientes que Saíram')
plt.ylabel('')
plt.show()


In [ ]:
#Histograma da idade dos clientes que saíram
plt.figure(figsize=(8,6))
sns.histplot(data=clientes_sairam, x='idade', bins=15, kde=True, color='coral')
plt.title('Distribuição de Idade dos Clientes que Saíram')
plt.xlabel('Idade')
plt.ylabel('Frequência')
plt.show()


In [ ]:
# Correlação entre variáveis numéricas
plt.figure(figsize=(10, 6))
sns.heatmap(df[colunas_numericas].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlação')
plt.show()
